# 본 파일을 실행하기 전에 "1_Display_Sensor_Anomaly_Analysis_데이터저장.ipynb" 을 실행시켜서 데이터를 저장하세요

   - 1_Display_Sensor_Anomaly_Analysis_결측치처리와 상관계수저장.ipynb 파일을 실행하고 나면 working_directory에 다음과 같은 파일이 생성됩니다

       \- 전체 데이터 : fact_data.csv

       \- sensor 데이터 : sensor_data.csv

       \- 폐기율 관련 데이터 : trash_data.sv

       \-  컬럼별 모든 상관계수정보 

       ```
       corr_df0-100.csv
       corr_df100-200.csv
       corr_df200-300.csv
       corr_df300-500.csv
       corr_df500-700.csv
       corr_df700-891.csv
       ```
       \- XX 이상의 상관계수 정보 :high_coef_df_upper_XX.csv


- colab에서 실행시 이 소스를 실행시키고 다시 runtime restart를 해주세요
- matplotlib의 한글 폰트 사용을 위한 소스입니다.

In [ ]:
!sudo apt-get install -y fonts-nanum
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 0s (41.1 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and di

In [ ]:
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
from scipy import stats
import seaborn as sns
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Display_Sensor_Anomaly_Analysis"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    
plt.rc('font', family='NanumBarunGothic') 

import warnings
warnings.filterwarnings(action='ignore')

np.random.seed(42)

In [ ]:
working_dir = '/content/drive/MyDrive/Colab Notebooks/data/Display_Sensor_Anomaly_Analysis/'

- 저장 데이터 불러오기

In [ ]:
fact_data = pd.read_csv(working_dir + 'fact_data.csv', index_col='dat...date.name.')
sensor_data = pd.read_csv(working_dir + 'sensor_data.csv', index_col='dat...date.name.')
trash_data = pd.read_csv(working_dir + 'trash_data.csv', index_col='dat...date.name.')

,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,X1WOS45105_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
dat...date.name.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.818900,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.730000,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
2016-01-01 10,-1.

- 상관계수 저장 파일 불러오기

In [ ]:
compare_num = 1.0

file_name = 'high_coef_df_upper_' + str(compare_num)

# csv 파일불러오기
high_coef_df = pd.read_csv(working_dir + file_name + '.csv',usecols=(1,2,3))
high_coef_df

,col1,col2,coef
0,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,1.0
1,DAMP.POSITION.4.BAY.CTRL.1TIC33424.PV,DAMP.POSITION.6.BAY.CTRL.1TIC33426.PV,1.0
2,ATM.FLOW.SECT.6.N2.H2.1FI33410.PV,ATM.FLOW.SECT.9.N2.H2.1FIC33416.PV,1.0
3,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,1.0
4,ATM.FLOW.SECT.6.N2.H2.1FI33410.PV,ATM.FLOW.SECT.10.N2.H2.1FIC33418.PV,1.0
...,...,...,...
97,LEHR.DRV.MTR..M11.2.CUR.1WII40119.PV,LEHR.DRV.MTR..M12.2.CUR.1WII40121.PV,1.0
98,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,TMP.TIN..BAY.4.LEFT.1TI30203.PV,1.0
99,LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,1.0
100,LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV,LEHR.DRV.MTR..M17.2.CUR.1WII40131.PV,1.0


## VIF계수 출력
- ⭐소요시간 약 40분이므로 csv파일 받아서 하기⭐

### 다중공선성 해결 방법
1. VIF(분산팽창요인)으로 다중공선성 확인
2. Feature Importance 확인
3. 회귀분석모델 수행후, 정확성과 loss함수 확인

- 다중공선성이 높다고 feature를 무조건 제거하면 안됨. 
- 두 컬럼의 평균값으로 대치한다???? => PCA





- VIF계수 출력 및 파일 저장

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 모든 sensor 컬럼에 대한 VIF계수를 구하고자 할때 주석해제
# columns_all = sensor_data.columns

#  일부 컬럼에 대해서  VIF계수를 구하고자 할때 주석해제
# drop할 컬럼명 넣가
# drop_columns = []
# columns_all = [col for col in columns_all if col not in drop_columns]


# 피처마다의 VIF 계수를 출력
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(fact_data[columns_all].values, i) for i in range(fact_data[columns_all].values.shape[1])]
vif["features"] = fact_data[columns_all].columns
vif

## 분산팽창 파일 저장
vifdf = vif.sort_values(by="VIF Factor",ascending=False).round(1)
vifdf.to_csv(working_dir + 'vifdf.csv')

- VIF계수 파일 불러오기

In [10]:
vifdf = pd.read_csv(working_dir + 'vifdf.csv', usecols=[1,2])
vifdf

,VIF Factor,features
0,7.901052e+13,LOWER.AB3.Z41.TMP.1TIC41356.PV
1,7.698461e+13,D_AB1_L_UL
2,6.169315e+13,DCS_Offset_RPM_M12.1WOS45112_PV
3,5.811096e+13,UPPER.AB1.Z2.TMP.1TIC41102.PV
4,5.267368e+13,UPPER.AB4.Z44.TMP.1TIC41402.PV
...,...,...
814,0.000000e+00,LEHR.DRV.MTR..M9.2.CUR.1WII40115.PV
815,0.000000e+00,TMP.TIN..BAY.4.LEFT.1TI30203.PV
816,0.000000e+00,LEHR.DRV.MTR..M4.2.CUR.1WII40105.PV
817,0.000000e+00,LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV


### VIF가 XX이상인 데이터만 추출해보기
- compare_num2를 변경후 실행하세요

In [11]:
compare_num2 = 10

high_vif_col = np.where(np.array(vifdf.iloc[:,0]) > compare_num2)

vifdf.iloc[:len(list(high_vif_col[0])),:]

,VIF Factor,features
0,7.901052e+13,LOWER.AB3.Z41.TMP.1TIC41356.PV
1,7.698461e+13,D_AB1_L_UL
2,6.169315e+13,DCS_Offset_RPM_M12.1WOS45112_PV
3,5.811096e+13,UPPER.AB1.Z2.TMP.1TIC41102.PV
4,5.267368e+13,UPPER.AB4.Z44.TMP.1TIC41402.PV
...,...,...
776,1.810000e+01,LOR.3.R.Axis
777,1.440000e+01,LOR.2.L.Axis
778,1.370000e+01,LIFT.OUT.ROL.MTR.M1.CUR.1WII40101.PV
779,1.370000e+01,ATM.FLOW.SECT.2.N2.1FI33401.PV


## 분산도

In [13]:
# 분산 var()
print(np.var(fact_data['LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101']))

1.0000475179378774
